In [4]:
import sys
import pandas as pd
import os
import sys
#!{sys.executable} -m install keplergl
#from keplergl import KeplerGl

import pickle
#!{sys.executable} -m install h3
from h3 import h3

def load_pickle(file_path):
    with open(file_path, 'rb') as handle:
        return pickle.load(handle)
        
def save_pickle(obj, filepath): 
    with open(filepath, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

Для корректной работы KeplerGl в jupyter notebook: https://docs.kepler.gl/docs/keplergl-jupyter

In [5]:
#os.makedirs('configs', exist_ok=True)

In [6]:
df_roads = pd.read_csv("train/roads_dataset.csv")
df_transport = pd.read_csv("train/routes_dataset.csv")
df_population = pd.read_csv("train/rosstat_population_all_cities.csv")
df_isochrones = pd.read_csv("train/isochrones_walk_dataset.csv")
df_stops = pd.read_csv("train/osm_stops.csv")
df_companies = pd.read_csv("train/osm_amenity.csv")

## Дороги
**highway** - тип дороги

**geometry** - shapely геометрия дороги

**oneway** - признак одностороннего движения

**name** - название улицы

**city** - город

In [7]:
df_roads.head()

,osmid,highway,geometry,oneway,name,city
0,33997807,primary_link,"LINESTRING (30.3314215 59.895094, 30.3311238 5...",False,NaN,Санкт-Петербург
1,201380605,primary,"LINESTRING (30.3314215 59.895094, 30.3318177 5...",True,Лиговский проспект,Санкт-Петербург
2,33997784,primary_link,"LINESTRING (30.3375246 59.8992885, 30.3379026 ...",False,NaN,Санкт-Петербург
3,178581714,primary,"LINESTRING (30.3375246 59.8992885, 30.3372282 ...",True,Лиговский проспект,Санкт-Петербург
4,4459876,primary,"LINESTRING (30.3395524 59.9017314, 30.3398299 ...",True,Воздухоплавательная улица,Санкт-Петербург


## Маршруты
**geometry** - shapely геометрия маршрута

**type** - тип общественного транспорта

**city** - город

**name** - идентификатор маршрута

In [8]:
df_transport.head()

,geometry,type,city,name
0,"LINESTRING (30.367515 60.012027, 30.367518 60....",tram,Санкт-Петербург,d645920e395fedad7bbbed0eca3fe2e0
1,"LINESTRING (30.231228 60.023206, 30.233159 60....",tram,Санкт-Петербург,b53b3a3d6ab90ce0268229151c9bde11
2,"LINESTRING (30.355657 59.956911, 30.355829 59....",tram,Санкт-Петербург,eccbc87e4b5ce2fe28308fd9f2a7baf3
3,"LINESTRING (30.235013 59.956634, 30.234953 59....",tram,Санкт-Петербург,1679091c5a880faf6fb5e6087eb1b2dc
4,"LINESTRING (30.355545 59.956849, 30.355398 59....",tram,Санкт-Петербург,37693cfc748049e45d87b8c7d8b9aacd


## Население
**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**population** - насление в хексагоне

**lat, lon** - широта/долгота центра хексагоны

In [25]:
df_population.head()

34855
33724


city
Екатеринбург       1823256
Нижний Новгород    1465501
Новосибирск        2002930
Санкт-Петербург    5158815
Name: population, dtype: int64

## Изохроны 

https://medium.com/@urbica/%D0%B3%D0%B0%D0%BB%D1%8C%D1%82%D0%BE%D0%BD-c7360acf1ec4

**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**walk_15min** - shapely геометрия 15 минутной пешей доступности из точки (lat, lon)

**lat, lon** - широта/долгота центра хексагоны

### Магазины-кампании и прочее 
**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**lat, lon** - широта/долгота центра хексагоны

**остальные колонки** - число организаций типа "название колонки" в хексагоне

In [11]:
df_companies.head()

,geo_h3_10,city,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,Женская одежда,...,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat,lon
0,8a0bb0200a5ffff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,54.953869,83.163008
1,8a0bb0200b37fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,54.948563,83.153038
2,8a0bb0201cd7fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,54.958064,83.174064
3,8a0bb0201d97fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,54.953799,83.167407
4,8a0bb0204027fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.935864,83.126481


## Остановки
**osm_id** - идентификатор 

**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**lat, lon** - широта/долгота центра хексагоны

**type** - тип остановки

**city** - город

**name** - название остановки

In [12]:
df_stops.head()

,osm_id,name,lat,lon,type,city,geo_h3_10
0,192192625,Ж/Д Вокзал,56.856321,60.605703,tram_stop,Екатеринбург,8a10c228a68ffff
1,192192653,Кондукторская,56.859895,60.627218,tram_stop,Екатеринбург,8a10dc924177fff
2,192192654,Учителей,56.857951,60.630261,tram_stop,Екатеринбург,8a10dc92402ffff
3,192196917,Новгородцевой,56.836949,60.680253,tram_stop,Екатеринбург,8a10dc920697fff
4,192196926,Сиреневый бульвар,56.835040,60.685064,tram_stop,Екатеринбург,8a10dc923d17fff


### Визуализируем данные

In [13]:
map_config_path = "configs/map-config.pkl"
if os.path.isfile(map_config_path):
    config = load_pickle(map_config_path)
else:
    config = {}

In [14]:
map_kepler = KeplerGl(height = 750, config=config)
map_kepler.add_data(df_roads, 'дороги')
map_kepler.add_data(df_transport, 'маршруты транспорта')
map_kepler.add_data(df_population.drop('geo_h3_10', axis = 1), 'Население')
map_kepler.add_data(df_isochrones.drop('geo_h3_10', axis = 1), 'Пешая доступность')
map_kepler.add_data(df_stops.drop('geo_h3_10', axis = 1), 'Остановки')
map_kepler

NameError: name 'KeplerGl' is not defined

In [ ]:
save_pickle(map_kepler.config, "configs/map-config.pkl")

## 1. Население  

In [28]:
print(len(df_population))
#Why???

print(len(df_population['geo_h3_10'].unique()))
print(len(df_population.drop_duplicates()))
df_population_by_city = df_population.drop_duplicates().groupby('city').sum()['population']
df_population_by_city

34855
33724
33724


city
Екатеринбург       1787172
Нижний Новгород    1465501
Новосибирск        2002930
Санкт-Петербург    5158815
Name: population, dtype: int64

###  Плотность жилой постройки

In [37]:
#print(df_companies.head())
feature_cols = list(df_companies)
feature_cols.remove('geo_h3_10')
feature_cols.remove('city')
feature_cols.remove('lat')
feature_cols.remove('lon')
print(feature_cols)
df_companies.fillna(0)
df_total_bulding_by_city = df_companies.drop_duplicates().groupby('city')[feature_cols].agg('sum')
df_total_bulding_by_city

['Автозапчасти для иномарок', 'Авторемонт и техобслуживание (СТО)', 'Алкогольные напитки', 'Аптеки', 'Банки', 'Быстрое питание', 'Доставка готовых блюд', 'Женская одежда', 'Кафе', 'Косметика / Парфюмерия', 'Ногтевые студии', 'Овощи / Фрукты', 'Парикмахерские', 'Платёжные терминалы', 'Постаматы', 'Продуктовые магазины', 'Пункты выдачи интернет-заказов', 'Рестораны', 'Страхование', 'Супермаркеты', 'Цветы', 'Шиномонтаж']


,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,Женская одежда,Кафе,Косметика / Парфюмерия,...,Парикмахерские,Платёжные терминалы,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж
city,,,,,,,,,,,,,,,,,,,,,
Екатеринбург,732.0,1807.0,940.0,1596.0,428.0,941.0,454.0,2352.0,664.0,795.0,...,1698.0,1324.0,1050.0,594.0,859.0,313.0,405.0,2147.0,534.0,454.0
Нижний Новгород,500.0,1318.0,487.0,1521.0,408.0,697.0,441.0,1812.0,658.0,530.0,...,1527.0,1124.0,1007.0,1327.0,859.0,222.0,356.0,1493.0,423.0,449.0
Новосибирск,837.0,2793.0,474.0,1968.0,417.0,1252.0,566.0,2273.0,802.0,1010.0,...,2431.0,2058.0,926.0,1542.0,654.0,240.0,479.0,2189.0,499.0,458.0
Санкт-Петербург,1350.0,4561.0,1802.0,4620.0,1488.0,4070.0,929.0,6907.0,2702.0,2228.0,...,6436.0,4845.0,2890.0,2833.0,2908.0,2079.0,1406.0,5860.0,1670.0,1545.0


## 3. Проходимость локации  

In [42]:
def convert_to_poly(string):
    return shapely.wkt.loads(string)
def get_area(polygon):
    return polygon.area

In [45]:
df_isochrones.head()
#len(df_isochrones['walk_15min'][13])
import shapely.wkt
walking_ploygones = df_isochrones['walk_15min']
Polygones = walking_ploygones.apply(convert_to_poly)
df_isochrones['area'] = Polygones.apply(get_area)
df_isochrones['area'][:10]
df_with_areas = df_isochrones.copy()
df_with_areas

,city,walk_15min,geo_h3_10,lat,lon,area
0,Нижний Новгород,"POLYGON ((43.776762 56.391904, 43.775762 56.39...",8a101601044ffff,56.383627,43.777762,0.000259
1,Нижний Новгород,"POLYGON ((43.773717 56.39169, 43.772717 56.390...",8a101601045ffff,56.382411,43.777717,0.000237
2,Нижний Новгород,"POLYGON ((43.773458 56.388816, 43.770682 56.38...",8a10160104cffff,56.380606,43.779458,0.000228
3,Нижний Новгород,"POLYGON ((43.788412 56.382862, 43.784155 56.38...",8a10160104dffff,56.379391,43.779412,0.000158
4,Нижний Новгород,"POLYGON ((43.775051 56.391224, 43.773051 56.39...",8a1016010607fff,56.384913,43.785051,0.000268
...,...,...,...,...,...,...
47092,Екатеринбург,"POLYGON ((60.595314 56.77692, 60.594258 56.776...",8a10dc936d97fff,56.767846,60.585314,0.000345
47093,Екатеринбург,"POLYGON ((60.596524 56.777762, 60.594996 56.77...",8a10dc936d9ffff,56.768217,60.587524,0.000382
47094,Екатеринбург,"POLYGON ((60.596222 56.777785, 60.595956 56.77...",8a10dc936da7fff,56.769914,60.584222,0.000343
47095,Екатеринбург,"POLYGON ((60.597432 56.778491, 60.596432 56.77...",8a10dc936daffff,56.770285,60.586432,0.000383


### Сортировка по площади и отображение:  

In [48]:
df_with_areas.sort_values(['city','area'],ascending=False)
df_with_areas[['city','area']]

,city,area
0,Нижний Новгород,0.000259
1,Нижний Новгород,0.000237
2,Нижний Новгород,0.000228
3,Нижний Новгород,0.000158
4,Нижний Новгород,0.000268
...,...,...
47092,Екатеринбург,0.000345
47093,Екатеринбург,0.000382
47094,Екатеринбург,0.000343
47095,Екатеринбург,0.000383
